# 🔐 Smart Contract Vulnerability Detection Training
## Specialized ML for Solidity Security Analysis

This notebook trains advanced ML models specifically for detecting vulnerabilities in smart contracts.

**Features:**
- 🔍 12 Critical Smart Contract Vulnerability Types
- 🧠 Advanced Solidity Code Feature Extraction
- ⚡ GPU-Accelerated XGBoost & Neural Networks
- 🎯 Real-time Vulnerability & Severity Prediction
- 📊 DeFi Protocol-Specific Training Data

**Supported Vulnerabilities:**
- Reentrancy Attacks
- Integer Overflow/Underflow
- Unchecked External Calls
- Access Control Issues
- Timestamp Dependence
- And 7 more critical patterns...

---

## 🔧 Setup & Dependencies

In [ ]:
# Check GPU for smart contract training
import torch
print(f"🔥 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print("⚡ Ready for GPU-accelerated smart contract analysis!")
else:
    print("⚠️ No GPU detected. Training will be slower but still functional.")

In [ ]:
# Install specialized dependencies for smart contract analysis
!pip install -q scikit-learn pandas numpy matplotlib seaborn
!pip install -q xgboost lightgbm
!pip install -q plotly kaleido
!pip install -q joblib
!pip install -q networkx  # For code graph analysis
!pip install -q textstat  # For code complexity metrics

print("✅ Smart contract analysis dependencies installed!")

In [ ]:
# Mount Google Drive for smart contract models
from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs('/content/drive/MyDrive/SmartContract_Models', exist_ok=True)
os.makedirs('/content/drive/MyDrive/SmartContract_Results', exist_ok=True)
os.makedirs('/content/drive/MyDrive/SmartContract_Datasets', exist_ok=True)

print("✅ Google Drive mounted - Smart contract workspace ready!")

## 📦 Smart Contract Analysis Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import warnings
from datetime import datetime
from pathlib import Path
import logging
import hashlib
from collections import Counter

# ML imports for smart contract analysis
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    classification_report, confusion_matrix, roc_auc_score
)
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import joblib

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("viridis")

print("✅ Smart contract analysis libraries loaded!")

## 🤖 Smart Contract Vulnerability Trainer

In [ ]:
class SmartContractVulnTrainer:
    """Advanced smart contract vulnerability detection trainer for Colab"""

    def __init__(self, drive_path='/content/drive/MyDrive/SmartContract_Models'):
        self.drive_path = Path(drive_path)
        self.results_path = Path('/content/drive/MyDrive/SmartContract_Results')
        self.dataset_path = Path('/content/drive/MyDrive/SmartContract_Datasets')
        
        # Setup logging
        self.logger = self._setup_logging()
        
        # Model storage
        self.models = {}
        self.vectorizers = {}
        self.encoders = {}
        self.scalers = {}
        
        # Training results
        self.training_results = {}
        
        # GPU device
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # Smart contract vulnerability categories
        self.vulnerability_categories = {
            'reentrancy': {
                'severity': 'Critical',
                'patterns': ['call.value', 'send(', 'transfer(', 'external call'],
                'description': 'Recursive calling vulnerability allowing state manipulation',
                'bounty_multiplier': 3.0
            },
            'integer_overflow': {
                'severity': 'High',
                'patterns': ['++', '--', 'SafeMath', 'unchecked', 'overflow'],
                'description': 'Arithmetic operations that can exceed variable limits',
                'bounty_multiplier': 2.5
            },
            'unchecked_call': {
                'severity': 'Medium',
                'patterns': ['call(', '.call', 'delegatecall', 'staticcall'],
                'description': 'External calls without proper return value checking',
                'bounty_multiplier': 1.8
            },
            'access_control': {
                'severity': 'High',
                'patterns': ['onlyOwner', 'modifier', 'require', 'msg.sender'],
                'description': 'Improper access restrictions and authorization',
                'bounty_multiplier': 2.3
            },
            'timestamp_dependence': {
                'severity': 'Medium',
                'patterns': ['block.timestamp', 'now', 'block.number'],
                'description': 'Logic dependent on manipulable blockchain timestamps',
                'bounty_multiplier': 1.5
            },
            'tx_origin': {
                'severity': 'Medium',
                'patterns': ['tx.origin', 'msg.sender'],
                'description': 'Using tx.origin for authorization instead of msg.sender',
                'bounty_multiplier': 1.6
            },
            'uninitialized_storage': {
                'severity': 'High',
                'patterns': ['storage', 'uninitialized', 'pointer'],
                'description': 'Storage pointers that are not properly initialized',
                'bounty_multiplier': 2.2
            },
            'delegatecall_injection': {
                'severity': 'Critical',
                'patterns': ['delegatecall', 'callcode', 'assembly'],
                'description': 'Unsafe use of delegatecall allowing code injection',
                'bounty_multiplier': 3.2
            },
            'short_address': {
                'severity': 'Medium',
                'patterns': ['msg.data.length', 'address', 'parameter'],
                'description': 'Missing validation of address parameter length',
                'bounty_multiplier': 1.4
            },
            'dos_gas_limit': {
                'severity': 'Medium',
                'patterns': ['gas', 'loop', 'array', 'unbounded'],
                'description': 'Operations that can exceed gas limits causing DoS',
                'bounty_multiplier': 1.7
            },
            'random_weakness': {
                'severity': 'High',
                'patterns': ['random', 'blockhash', 'keccak256', 'predictable'],
                'description': 'Weak randomness that can be predicted or manipulated',
                'bounty_multiplier': 2.1
            },
            'front_running': {
                'severity': 'Medium',
                'patterns': ['mempool', 'transaction', 'order', 'MEV'],
                'description': 'Susceptible to transaction ordering manipulation',
                'bounty_multiplier': 1.9
            }
        }
        
    def _setup_logging(self):
        """Setup logging for Colab environment"""
        logger = logging.getLogger('SmartContractVuln_Colab')
        logger.setLevel(logging.INFO)
        
        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            
        return logger

print("✅ Smart Contract Vulnerability Trainer class initialized!")

## 📊 Smart Contract Dataset Generation

In [ ]:
# Add data generation methods to the trainer
def generate_smart_contract_dataset(self, n_samples=8000):
    """Generate comprehensive smart contract vulnerability dataset"""
    self.logger.info(f"🔐 Generating {n_samples:,} smart contract vulnerability samples...")
    
    # DeFi protocols by tier
    defi_protocols = {
        'tier1': {
            'protocols': ['Uniswap', 'Compound', 'Aave', 'MakerDAO', 'Curve'],
            'bounty_base': (100000, 1000000),
            'weight': 0.3
        },
        'tier2': {
            'protocols': ['SushiSwap', 'Yearn', 'Synthetix', 'Balancer', '1inch'],
            'bounty_base': (50000, 500000),
            'weight': 0.4
        },
        'tier3': {
            'protocols': ['PancakeSwap', 'dYdX', 'Convex', 'Frax', 'Liquity'],
            'bounty_base': (25000, 250000),
            'weight': 0.3
        }
    }
    
    contract_types = {
        'token': ['ERC20', 'ERC721', 'ERC1155'],
        'defi': ['DEX', 'AMM', 'Pool', 'Vault', 'Strategy'],
        'infrastructure': ['Proxy', 'Factory', 'Router', 'Oracle'],
        'governance': ['Governor', 'Timelock', 'Voting']
    }
    
    all_data = []
    
    for i in range(n_samples):
        # Select vulnerability type
        vuln_type = np.random.choice(list(self.vulnerability_categories.keys()))
        vuln_info = self.vulnerability_categories[vuln_type]
        
        # Select protocol tier
        tier_names = list(defi_protocols.keys())
        tier_weights = [defi_protocols[t]['weight'] for t in tier_names]
        selected_tier = np.random.choice(tier_names, p=tier_weights)
        
        tier_data = defi_protocols[selected_tier]
        protocol = np.random.choice(tier_data['protocols'])
        
        # Select contract type
        contract_category = np.random.choice(list(contract_types.keys()))
        contract_type = np.random.choice(contract_types[contract_category])
        
        # Generate vulnerable code
        code_snippet = self._generate_vulnerable_code(vuln_type, vuln_info)
        
        # Calculate bounty
        base_min, base_max = tier_data['bounty_base']
        base_bounty = np.random.uniform(base_min, base_max)
        final_bounty = base_bounty * vuln_info['bounty_multiplier'] * np.random.uniform(0.7, 1.4)
        
        # Extract code features
        code_features = self._extract_advanced_code_features(code_snippet)
        
        record = {
            'id': f"sc_{i+1}",
            'vulnerability_type': vuln_type,
            'severity_level': vuln_info['severity'],
            'protocol': protocol,
            'protocol_tier': selected_tier,
            'contract_type': contract_type,
            'contract_category': contract_category,
            'code_snippet': code_snippet,
            'bounty_amount': round(final_bounty, 2),
            'description': f"{vuln_info['severity']} {vuln_type} in {protocol} {contract_type}",
            'cve_score': self._generate_cve_score(vuln_info['severity']),
            **code_features
        }
        
        all_data.append(record)
    
    df = pd.DataFrame(all_data)
    
    # Save to Google Drive
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_path = self.dataset_path / f"smart_contract_vulns_{timestamp}.csv"
    df.to_csv(csv_path, index=False)
    
    self.logger.info(f"✅ Generated {len(df):,} samples")
    self.logger.info(f"💰 Bounty range: ${df['bounty_amount'].min():,.0f} - ${df['bounty_amount'].max():,.0f}")
    
    return df

def _generate_vulnerable_code(self, vuln_type, vuln_info):
    """Generate realistic vulnerable Solidity code"""
    
    code_templates = {
        'reentrancy': '''
contract VulnerableBank {
    mapping(address => uint256) public balances;
    
    function withdraw(uint256 amount) external {
        require(balances[msg.sender] >= amount, "Insufficient balance");
        
        (bool success, ) = msg.sender.call{value: amount}("");
        require(success, "Transfer failed");
        
        balances[msg.sender] -= amount;  // State change after external call!
    }
    
    function deposit() external payable {
        balances[msg.sender] += msg.value;
    }
}
        ''',
        
        'integer_overflow': '''
contract VulnerableToken {
    mapping(address => uint256) public balances;
    uint256 public totalSupply;
    
    function transfer(address to, uint256 value) external returns (bool) {
        balances[msg.sender] -= value;  // No SafeMath - underflow risk!
        balances[to] += value;          // No SafeMath - overflow risk!
        
        emit Transfer(msg.sender, to, value);
        return true;
    }
    
    event Transfer(address indexed from, address indexed to, uint256 value);
}
        ''',
        
        'delegatecall_injection': '''
contract VulnerableProxy {
    address public implementation;
    address public admin;
    
    function upgrade(address newImplementation) external {
        require(msg.sender == admin, "Only admin");
        implementation = newImplementation;
    }
    
    fallback() external payable {
        address impl = implementation;
        assembly {
            calldatacopy(0, 0, calldatasize())
            let result := delegatecall(gas(), impl, 0, calldatasize(), 0, 0)
            returndatacopy(0, 0, returndatasize())
            
            switch result
            case 0 { revert(0, returndatasize()) }
            default { return(0, returndatasize()) }
        }
    }
}
        ''',
        
        'access_control': '''
contract VulnerableVault {
    address public owner;
    mapping(address => uint256) public deposits;
    
    function changeOwner(address newOwner) external {
        owner = newOwner;  // No access control!
    }
    
    function emergencyWithdraw() external {
        payable(msg.sender).transfer(address(this).balance);  // Anyone can drain!
    }
    
    function deposit() external payable {
        deposits[msg.sender] += msg.value;
    }
}
        ''',
        
        'timestamp_dependence': '''
contract VulnerableLottery {
    uint256 public lastDrawTime;
    mapping(address => uint256) public tickets;
    
    function buyTicket() external payable {
        require(msg.value >= 0.1 ether, "Minimum bet");
        tickets[msg.sender] += msg.value;
    }
    
    function draw() external {
        require(block.timestamp > lastDrawTime + 1 hours, "Too early");
        
        // Weak randomness based on timestamp!
        uint256 winner = uint256(keccak256(abi.encodePacked(
            block.timestamp,
            block.difficulty
        ))) % 100;
        
        if (winner < 50) {
            payable(msg.sender).transfer(address(this).balance);
        }
        
        lastDrawTime = block.timestamp;
    }
}
        '''
    }
    
    return code_templates.get(vuln_type, '''
contract GenericVulnerable {
    mapping(address => uint256) public balances;
    
    function vulnerableFunction(uint256 value) external {
        balances[msg.sender] += value;
    }
}
    ''')

def _extract_advanced_code_features(self, code):
    """Extract comprehensive features from Solidity code"""
    features = {
        'line_count': len(code.split('\n')),
        'function_count': len(re.findall(r'function\s+\w+', code)),
        'modifier_count': len(re.findall(r'modifier\s+\w+', code)),
        'external_calls': len(re.findall(r'\.call\(|\.delegatecall\(|\.send\(|\.transfer\(', code)),
        'state_variables': len(re.findall(r'(mapping|uint256|address|bool)\s+(public|private|internal)', code)),
        'require_statements': len(re.findall(r'require\(', code)),
        'assembly_blocks': len(re.findall(r'assembly\s*\{', code)),
        'event_declarations': len(re.findall(r'event\s+\w+', code)),
        'payable_functions': len(re.findall(r'payable', code)),
        'external_functions': len(re.findall(r'function\s+\w+.*external', code)),
        'public_functions': len(re.findall(r'function\s+\w+.*public', code)),
        'loops': len(re.findall(r'for\s*\(|while\s*\(', code)),
        'complexity_score': min(1.0, len(code.split('\n')) / 100.0),
        'has_fallback': 1 if 'fallback' in code else 0,
        'has_receive': 1 if 'receive' in code else 0,
        'uses_msg_sender': len(re.findall(r'msg\.sender', code)),
        'uses_block_timestamp': len(re.findall(r'block\.timestamp|now', code)),
        'uses_tx_origin': len(re.findall(r'tx\.origin', code))
    }
    
    return features

def _generate_cve_score(self, severity):
    """Generate CVE-like scores"""
    ranges = {
        'Critical': (9.0, 10.0),
        'High': (7.0, 8.9),
        'Medium': (4.0, 6.9),
        'Low': (0.1, 3.9)
    }
    min_s, max_s = ranges[severity]
    return round(np.random.uniform(min_s, max_s), 1)

# Add methods to the class
SmartContractVulnTrainer.generate_smart_contract_dataset = generate_smart_contract_dataset
SmartContractVulnTrainer._generate_vulnerable_code = _generate_vulnerable_code
SmartContractVulnTrainer._extract_advanced_code_features = _extract_advanced_code_features
SmartContractVulnTrainer._generate_cve_score = _generate_cve_score

print("✅ Smart contract data generation methods added!")

## 🏋️ Smart Contract Model Training

In [ ]:
# Add training methods for smart contract analysis
def prepare_smart_contract_features(self, df):
    """Prepare advanced features for smart contract ML"""
    self.logger.info("🔧 Preparing smart contract features...")
    
    features = []
    
    for _, row in df.iterrows():
        # Create comprehensive feature vector
        feature_vector = [
            # Code complexity features
            row['line_count'],
            row['function_count'], 
            row['modifier_count'],
            row['external_calls'],
            row['state_variables'],
            row['require_statements'],
            row['assembly_blocks'],
            row['event_declarations'],
            row['payable_functions'],
            row['external_functions'],
            row['public_functions'],
            row['loops'],
            row['complexity_score'],
            row['cve_score'],
            
            # Protocol tier features
            1 if row['protocol_tier'] == 'tier1' else 0,
            1 if row['protocol_tier'] == 'tier2' else 0,
            1 if row['protocol_tier'] == 'tier3' else 0,
            
            # Contract type features
            1 if row['contract_category'] == 'token' else 0,
            1 if row['contract_category'] == 'defi' else 0,
            1 if row['contract_category'] == 'infrastructure' else 0,
            1 if row['contract_category'] == 'governance' else 0,
            
            # Vulnerability pattern features
            row['has_fallback'],
            row['has_receive'],
            row['uses_msg_sender'],
            row['uses_block_timestamp'],
            row['uses_tx_origin'],
            
            # Security indicators
            row['require_statements'] / max(row['function_count'], 1),  # Security ratio
            row['external_calls'] / max(row['function_count'], 1),     # External call ratio
            1 if row['assembly_blocks'] > 0 else 0,                   # Uses assembly
            
            # Advanced features
            len(str(row['code_snippet'])),  # Code length
            row['function_count'] / max(row['line_count'], 1),  # Function density
        ]
        
        features.append(feature_vector)
    
    return np.array(features)

def train_vulnerability_classifier(self, X, y):
    """Train multi-class smart contract vulnerability classifier"""
    self.logger.info("🤖 Training smart contract vulnerability classifier...")
    
    # Label encoding
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    self.encoders['vulnerability'] = label_encoder
    
    # Feature scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    self.scalers['vulnerability'] = scaler
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    # Advanced model ensemble optimized for smart contracts
    models = {
        'xgboost_gpu': xgb.XGBClassifier(
            n_estimators=300,
            max_depth=10,
            learning_rate=0.08,
            subsample=0.85,
            colsample_bytree=0.85,
            random_state=42,
            eval_metric='mlogloss',
            tree_method='hist',
            device='cuda' if torch.cuda.is_available() else 'cpu'
        ),
        'random_forest': RandomForestClassifier(
            n_estimators=250,
            max_depth=15,
            min_samples_split=3,
            min_samples_leaf=1,
            random_state=42,
            n_jobs=-1
        ),
        'neural_network': MLPClassifier(
            hidden_layer_sizes=(512, 256, 128, 64),
            activation='relu',
            solver='adam',
            alpha=0.0001,
            learning_rate='adaptive',
            max_iter=800,
            random_state=42
        ),
        'gradient_boosting': GradientBoostingClassifier(
            n_estimators=200,
            max_depth=10,
            learning_rate=0.08,
            min_samples_split=8,
            random_state=42
        )
    }
    
    best_model = None
    best_score = -np.inf
    model_results = {}
    
    for name, model in models.items():
        self.logger.info(f"  Training {name}...")
        
        # Stratified cross-validation
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(model, X_train, y_train, cv=skf, scoring='accuracy', n_jobs=-1)
        
        # Train and evaluate
        model.fit(X_train, y_train)
        y_pred_test = model.predict(X_test)
        
        test_accuracy = accuracy_score(y_test, y_pred_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_test, average='weighted')
        
        model_results[name] = {
            'cv_accuracy_mean': cv_scores.mean(),
            'cv_accuracy_std': cv_scores.std(),
            'test_accuracy': test_accuracy,
            'test_precision': precision,
            'test_recall': recall,
            'test_f1': f1
        }
        
        self.logger.info(f"    CV: {cv_scores.mean():.3f}±{cv_scores.std():.3f}")
        self.logger.info(f"    Test: Acc={test_accuracy:.3f}, F1={f1:.3f}")
        
        if cv_scores.mean() > best_score:
            best_score = cv_scores.mean()
            best_model = model
    
    self.models['vulnerability_classifier'] = best_model
    
    # Final evaluation
    y_pred_final = best_model.predict(X_test)
    class_report = classification_report(
        y_test, y_pred_final,
        target_names=label_encoder.classes_,
        output_dict=True
    )
    
    results = {
        'best_model': type(best_model).__name__,
        'cv_accuracy_mean': best_score,
        'test_accuracy': accuracy_score(y_test, y_pred_final),
        'classification_report': class_report,
        'class_names': label_encoder.classes_.tolist(),
        'model_comparison': model_results,
        'samples_count': len(X),
        'features_count': X.shape[1]
    }
    
    self.training_results['vulnerability_classification'] = results
    self.logger.info(f"✅ Best vulnerability classifier: {results['best_model']} (Acc={best_score:.3f})")
    
    return results

def train_severity_predictor(self, df):
    """Train severity prediction for smart contracts"""
    self.logger.info("🎯 Training smart contract severity predictor...")
    
    # Prepare text features from code
    code_texts = []
    severities = []
    
    for _, row in df.iterrows():
        # Enhanced text representation
        text = f"{row['code_snippet']} {row['description']} {row['vulnerability_type']} {row['contract_type']}"
        code_texts.append(text)
        severities.append(row['severity_level'])
    
    # Advanced text vectorization for code
    vectorizer = TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 5),  # Capture larger code patterns
        stop_words=None,     # Keep all technical terms
        min_df=2,
        max_df=0.98,
        token_pattern=r'\b\w+\b'
    )
    
    X_text = vectorizer.fit_transform(code_texts)
    self.vectorizers['severity'] = vectorizer
    
    # Encode severity
    severity_encoder = LabelEncoder()
    y_severity = severity_encoder.fit_transform(severities)
    self.encoders['severity'] = severity_encoder
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_text.toarray(), y_severity, test_size=0.2, random_state=42, stratify=y_severity
    )
    
    # GPU-optimized model for severity
    model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
        eval_metric='mlogloss',
        tree_method='hist',
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    
    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train, y_train, cv=skf, scoring='accuracy')
    
    # Train final model
    model.fit(X_train, y_train)
    self.models['severity_predictor'] = model
    
    # Evaluation
    y_pred_test = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    
    class_report = classification_report(
        y_test, y_pred_test,
        target_names=severity_encoder.classes_,
        output_dict=True
    )
    
    results = {
        'model_type': type(model).__name__,
        'cv_accuracy_mean': cv_scores.mean(),
        'cv_accuracy_std': cv_scores.std(),
        'test_accuracy': test_accuracy,
        'classification_report': class_report,
        'class_names': severity_encoder.classes_.tolist(),
        'samples_count': len(X_text.toarray()),
        'features_count': X_text.shape[1]
    }
    
    self.training_results['severity_prediction'] = results
    self.logger.info(f"✅ Severity predictor: {results['model_type']} (Acc={cv_scores.mean():.3f})")
    
    return results

def save_models(self):
    """Save all models to Google Drive"""
    self.logger.info("💾 Saving smart contract models...")
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save models
    for model_name, model in self.models.items():
        model_path = self.drive_path / f"{model_name}_{timestamp}.pkl"
        joblib.dump(model, model_path)
        self.logger.info(f"  Saved {model_name}")
    
    # Save preprocessors
    for scaler_name, scaler in self.scalers.items():
        scaler_path = self.drive_path / f"scaler_{scaler_name}_{timestamp}.pkl"
        joblib.dump(scaler, scaler_path)
    
    for vec_name, vectorizer in self.vectorizers.items():
        vec_path = self.drive_path / f"vectorizer_{vec_name}_{timestamp}.pkl"
        joblib.dump(vectorizer, vec_path)
    
    for enc_name, encoder in self.encoders.items():
        enc_path = self.drive_path / f"encoder_{enc_name}_{timestamp}.pkl"
        joblib.dump(encoder, enc_path)
    
    # Save results
    results_path = self.results_path / f"smart_contract_results_{timestamp}.json"
    with open(results_path, 'w') as f:
        json.dump(self.training_results, f, indent=2, default=str)
    
    self.logger.info(f"✅ All models saved with timestamp: {timestamp}")
    return timestamp

# Add methods to the class
SmartContractVulnTrainer.prepare_smart_contract_features = prepare_smart_contract_features
SmartContractVulnTrainer.train_vulnerability_classifier = train_vulnerability_classifier
SmartContractVulnTrainer.train_severity_predictor = train_severity_predictor
SmartContractVulnTrainer.save_models = save_models

print("✅ Smart contract training methods added!")

## 🚀 Execute Smart Contract Training

In [ ]:
# Initialize smart contract trainer
print("🔐 Initializing Smart Contract Vulnerability Trainer...")
sc_trainer = SmartContractVulnTrainer()

# Generate comprehensive smart contract dataset
print("\n📊 Generating Smart Contract Vulnerability Dataset...")
print("=" * 60)
sc_df = sc_trainer.generate_smart_contract_dataset(n_samples=8000)

print(f"\n✅ Dataset Overview:")
print(f"Total samples: {len(sc_df):,}")
print(f"Bounty range: ${sc_df['bounty_amount'].min():,.0f} - ${sc_df['bounty_amount'].max():,.0f}")
print(f"Average bounty: ${sc_df['bounty_amount'].mean():,.0f}")
print(f"\n🔐 Vulnerability distribution:")
print(sc_df['vulnerability_type'].value_counts())
print(f"\n🎯 Severity distribution:")
print(sc_df['severity_level'].value_counts())
print(f"\n🏢 Protocol tier distribution:")
print(sc_df['protocol_tier'].value_counts())
print(f"\n📝 Contract type distribution:")
print(sc_df['contract_type'].value_counts())

In [ ]:
# Create comprehensive smart contract visualization
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Vulnerability Types Distribution',
        'Bounty by Severity Level',
        'Protocol Tier vs Bounty Amount',
        'Code Complexity vs CVE Score',
        'External Calls Distribution',
        'Function Count vs Line Count'
    ),
    specs=[[{"type": "bar"}, {"type": "box"}],
           [{"type": "scatter"}, {"type": "scatter"}],
           [{"type": "histogram"}, {"type": "scatter"}]]
)

# Vulnerability types distribution
vuln_counts = sc_df['vulnerability_type'].value_counts()
fig.add_trace(
    go.Bar(
        x=vuln_counts.index,
        y=vuln_counts.values,
        text=vuln_counts.values,
        textposition='auto',
        showlegend=False
    ),
    row=1, col=1
)

# Bounty by severity
for severity in sc_df['severity_level'].unique():
    severity_data = sc_df[sc_df['severity_level'] == severity]['bounty_amount']
    fig.add_trace(
        go.Box(y=severity_data, name=severity, showlegend=False),
        row=1, col=2
    )

# Protocol tier vs bounty
tier_colors = {'tier1': 'red', 'tier2': 'blue', 'tier3': 'green'}
for tier in sc_df['protocol_tier'].unique():
    tier_data = sc_df[sc_df['protocol_tier'] == tier]
    fig.add_trace(
        go.Scatter(
            x=tier_data['cve_score'],
            y=tier_data['bounty_amount'],
            mode='markers',
            name=tier,
            marker=dict(color=tier_colors[tier]),
            showlegend=False,
            opacity=0.7
        ),
        row=2, col=1
    )

# Code complexity vs CVE score
fig.add_trace(
    go.Scatter(
        x=sc_df['complexity_score'],
        y=sc_df['cve_score'],
        mode='markers',
        text=sc_df['vulnerability_type'],
        showlegend=False,
        opacity=0.6
    ),
    row=2, col=2
)

# External calls distribution
fig.add_trace(
    go.Histogram(x=sc_df['external_calls'], nbinsx=15, showlegend=False),
    row=3, col=1
)

# Function count vs line count
fig.add_trace(
    go.Scatter(
        x=sc_df['function_count'],
        y=sc_df['line_count'],
        mode='markers',
        text=sc_df['contract_type'],
        showlegend=False,
        opacity=0.6
    ),
    row=3, col=2
)

fig.update_layout(
    height=1200,
    title_text="Smart Contract Vulnerability Dataset Analysis",
    title_x=0.5
)

# Update axes
fig.update_xaxes(title_text="Vulnerability Type", tickangle=-45, row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Bounty Amount ($)", type="log", row=1, col=2)
fig.update_xaxes(title_text="CVE Score", row=2, col=1)
fig.update_yaxes(title_text="Bounty Amount ($)", type="log", row=2, col=1)
fig.update_xaxes(title_text="Complexity Score", row=2, col=2)
fig.update_yaxes(title_text="CVE Score", row=2, col=2)
fig.update_xaxes(title_text="External Calls", row=3, col=1)
fig.update_yaxes(title_text="Frequency", row=3, col=1)
fig.update_xaxes(title_text="Function Count", row=3, col=2)
fig.update_yaxes(title_text="Line Count", row=3, col=2)

fig.show()

# Display sample vulnerable code
print("\n🔍 Sample Vulnerable Smart Contract Code:")
print("=" * 60)
sample_row = sc_df.sample(1).iloc[0]
print(f"Vulnerability: {sample_row['vulnerability_type']}")
print(f"Severity: {sample_row['severity_level']}")
print(f"Protocol: {sample_row['protocol']} ({sample_row['protocol_tier']})")
print(f"Contract Type: {sample_row['contract_type']}")
print(f"Bounty: ${sample_row['bounty_amount']:,.0f}")
print(f"\nCode Snippet:")
print(sample_row['code_snippet'][:500] + "..." if len(sample_row['code_snippet']) > 500 else sample_row['code_snippet'])

In [ ]:
# Prepare features for smart contract training
print("🔧 Preparing Smart Contract Features...")
print("=" * 50)

X_sc = sc_trainer.prepare_smart_contract_features(sc_df)
y_sc = sc_df['vulnerability_type'].values

print(f"Feature matrix shape: {X_sc.shape}")
print(f"Target vector shape: {y_sc.shape}")
print(f"Feature range: [{X_sc.min():.2f}, {X_sc.max():.2f}]")
print(f"Unique vulnerabilities: {len(np.unique(y_sc))}")

# Feature importance analysis
feature_names = [
    'line_count', 'function_count', 'modifier_count', 'external_calls',
    'state_variables', 'require_statements', 'assembly_blocks', 'event_declarations',
    'payable_functions', 'external_functions', 'public_functions', 'loops',
    'complexity_score', 'cve_score', 'tier1', 'tier2', 'tier3',
    'token_type', 'defi_type', 'infra_type', 'gov_type',
    'has_fallback', 'has_receive', 'uses_msg_sender', 'uses_block_timestamp', 'uses_tx_origin',
    'security_ratio', 'external_call_ratio', 'uses_assembly',
    'code_length', 'function_density'
]

print(f"\n📊 Smart Contract Feature Summary:")
for i, feature in enumerate(feature_names):
    if i < X_sc.shape[1]:
        mean_val = X_sc[:, i].mean()
        std_val = X_sc[:, i].std()
        print(f"  {feature}: {mean_val:.3f} ± {std_val:.3f}")

print(f"\n✅ Smart contract features prepared successfully!")

In [ ]:
# Train vulnerability classifier
print("🤖 Training Smart Contract Vulnerability Classifier...")
print("=" * 60)

vuln_results = sc_trainer.train_vulnerability_classifier(X_sc, y_sc)

print(f"\n✅ Vulnerability Classifier Training Complete!")
print(f"Best Model: {vuln_results['best_model']}")
print(f"Cross-Validation Accuracy: {vuln_results['cv_accuracy_mean']:.3f}")
print(f"Test Accuracy: {vuln_results['test_accuracy']:.3f}")
print(f"Training Samples: {vuln_results['samples_count']:,}")
print(f"Features: {vuln_results['features_count']}")
print(f"Vulnerability Classes: {len(vuln_results['class_names'])}")

print(f"\n📊 Vulnerability Classes:")
for i, class_name in enumerate(vuln_results['class_names']):
    if class_name in vuln_results['classification_report']:
        metrics = vuln_results['classification_report'][class_name]
        print(f"  {class_name}: F1={metrics['f1-score']:.3f}, Precision={metrics['precision']:.3f}")

In [ ]:
# Train severity predictor
print("\n🎯 Training Smart Contract Severity Predictor...")
print("=" * 60)

severity_results = sc_trainer.train_severity_predictor(sc_df)

print(f"\n✅ Severity Predictor Training Complete!")
print(f"Model: {severity_results['model_type']}")
print(f"Cross-Validation Accuracy: {severity_results['cv_accuracy_mean']:.3f}")
print(f"Test Accuracy: {severity_results['test_accuracy']:.3f}")
print(f"Text Features: {severity_results['features_count']:,}")
print(f"Severity Classes: {', '.join(severity_results['class_names'])}")

print(f"\n📊 Severity Classification Performance:")
for class_name in severity_results['class_names']:
    if class_name in severity_results['classification_report']:
        metrics = severity_results['classification_report'][class_name]
        print(f"  {class_name}: F1={metrics['f1-score']:.3f}, Support={int(metrics['support'])}")

## 📋 Smart Contract Model Validation

In [ ]:
# Test with real-world smart contract scenarios
print("🧪 Testing Smart Contract Models with Real Scenarios...")
print("=" * 70)

test_contracts = [
    {
        'name': 'Reentrancy in DeFi Vault',
        'code': '''
function withdraw(uint256 amount) external {
    require(balances[msg.sender] >= amount);
    (bool success, ) = msg.sender.call{value: amount}("");
    require(success);
    balances[msg.sender] -= amount;
}
        ''',
        'expected_vuln': 'reentrancy',
        'expected_severity': 'Critical',
        'protocol': 'Compound',
        'contract_type': 'Vault'
    },
    {
        'name': 'Integer Overflow in Token',
        'code': '''
function transfer(address to, uint256 value) public returns (bool) {
    balances[msg.sender] -= value;
    balances[to] += value;
    return true;
}
        ''',
        'expected_vuln': 'integer_overflow',
        'expected_severity': 'High',
        'protocol': 'Uniswap',
        'contract_type': 'ERC20'
    },
    {
        'name': 'Delegatecall Injection',
        'code': '''
fallback() external payable {
    address impl = implementation;
    assembly {
        let result := delegatecall(gas(), impl, 0, calldatasize(), 0, 0)
        returndatacopy(0, 0, returndatasize())
        switch result
        case 0 { revert(0, returndatasize()) }
        default { return(0, returndatasize()) }
    }
}
        ''',
        'expected_vuln': 'delegatecall_injection',
        'expected_severity': 'Critical',
        'protocol': 'Aave',
        'contract_type': 'Proxy'
    },
    {
        'name': 'Access Control Issue',
        'code': '''
function changeOwner(address newOwner) external {
    owner = newOwner;
}

function emergencyWithdraw() external {
    payable(msg.sender).transfer(address(this).balance);
}
        ''',
        'expected_vuln': 'access_control',
        'expected_severity': 'High',
        'protocol': 'MakerDAO',
        'contract_type': 'Governor'
    }
]

successful_vuln_predictions = 0
successful_severity_predictions = 0
total_tests = len(test_contracts)

for i, test_case in enumerate(test_contracts, 1):
    print(f"\n🔍 Test {i}: {test_case['name']}")
    print(f"Expected: {test_case['expected_vuln']} ({test_case['expected_severity']})")
    
    try:
        # Extract features for this test case
        code_features = sc_trainer._extract_advanced_code_features(test_case['code'])
        
        # Create feature vector
        feature_vector = [
            code_features['line_count'],
            code_features['function_count'],
            code_features['modifier_count'],
            code_features['external_calls'],
            code_features['state_variables'],
            code_features['require_statements'],
            code_features['assembly_blocks'],
            code_features['event_declarations'],
            code_features['payable_functions'],
            code_features['external_functions'],
            code_features['public_functions'],
            code_features['loops'],
            code_features['complexity_score'],
            8.0,  # cve_score
            1 if test_case['protocol'] in ['Uniswap', 'Compound', 'Aave', 'MakerDAO'] else 0,  # tier1
            0, 0,  # tier2, tier3
            1 if test_case['contract_type'] in ['ERC20', 'ERC721'] else 0,  # token_type
            1 if test_case['contract_type'] in ['Vault', 'DEX'] else 0,  # defi_type
            1 if test_case['contract_type'] in ['Proxy'] else 0,  # infra_type
            1 if test_case['contract_type'] in ['Governor'] else 0,  # gov_type
            code_features['has_fallback'],
            code_features['has_receive'],
            code_features['uses_msg_sender'],
            code_features['uses_block_timestamp'],
            code_features['uses_tx_origin'],
            code_features['require_statements'] / max(code_features['function_count'], 1),  # security_ratio
            code_features['external_calls'] / max(code_features['function_count'], 1),  # external_call_ratio
            1 if code_features['assembly_blocks'] > 0 else 0,  # uses_assembly
            len(test_case['code']),  # code_length
            code_features['function_count'] / max(code_features['line_count'], 1),  # function_density
        ]
        
        # Predict vulnerability
        if 'vulnerability_classifier' in sc_trainer.models:
            features_scaled = sc_trainer.scalers['vulnerability'].transform([feature_vector])
            vuln_pred_encoded = sc_trainer.models['vulnerability_classifier'].predict(features_scaled)[0]
            vuln_prediction = sc_trainer.encoders['vulnerability'].inverse_transform([vuln_pred_encoded])[0]
            
            vuln_match = vuln_prediction == test_case['expected_vuln']
            if vuln_match:
                successful_vuln_predictions += 1
        else:
            vuln_prediction = 'unknown'
            vuln_match = False
        
        # Predict severity
        if 'severity_predictor' in sc_trainer.models:
            text = f"{test_case['code']} {test_case['name']} {test_case['contract_type']}"
            text_features = sc_trainer.vectorizers['severity'].transform([text])
            severity_pred_encoded = sc_trainer.models['severity_predictor'].predict(text_features.toarray())[0]
            severity_prediction = sc_trainer.encoders['severity'].inverse_transform([severity_pred_encoded])[0]
            
            severity_match = severity_prediction == test_case['expected_severity']
            if severity_match:
                successful_severity_predictions += 1
        else:
            severity_prediction = 'unknown'
            severity_match = False
        
        # Display results
        vuln_status = "✅" if vuln_match else "❌"
        sev_status = "✅" if severity_match else "❌"
        
        print(f"Predicted: {vuln_prediction} ({severity_prediction})")
        print(f"Results: Vulnerability {vuln_status} | Severity {sev_status}")
        
    except Exception as e:
        print(f"❌ Prediction failed: {e}")

# Calculate overall accuracy
vuln_accuracy = successful_vuln_predictions / total_tests
severity_accuracy = successful_severity_predictions / total_tests

print(f"\n🎯 SMART CONTRACT MODEL VALIDATION RESULTS:")
print(f"=" * 50)
print(f"Vulnerability Detection Accuracy: {vuln_accuracy:.1%} ({successful_vuln_predictions}/{total_tests})")
print(f"Severity Prediction Accuracy: {severity_accuracy:.1%} ({successful_severity_predictions}/{total_tests})")

if vuln_accuracy >= 0.75 and severity_accuracy >= 0.75:
    print("🟢 Smart contract models are performing excellently!")
elif vuln_accuracy >= 0.5 and severity_accuracy >= 0.5:
    print("🟡 Smart contract models show good performance")
else:
    print("🔴 Smart contract models need improvement")

## 💾 Save Models & Generate Deployment Code

In [ ]:
# Save all smart contract models
print("💾 Saving Smart Contract Models to Google Drive...")
timestamp = sc_trainer.save_models()

print(f"\n✅ All smart contract models saved successfully!")
print(f"📁 Models: /content/drive/MyDrive/SmartContract_Models/")
print(f"📁 Results: /content/drive/MyDrive/SmartContract_Results/")
print(f"🏷️ Timestamp: {timestamp}")

# Generate smart contract deployment code
deployment_code = f'''
# Smart Contract Vulnerability Detection Deployment
# Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
# Timestamp: {timestamp}

import joblib
import numpy as np
import re
from typing import Dict, Tuple

class SmartContractVulnDetector:
    def __init__(self, model_path_prefix):
        """Load trained smart contract vulnerability detection models"""
        
        # Load models
        self.vuln_classifier = joblib.load(f"{{model_path_prefix}}/vulnerability_classifier_{timestamp}.pkl")
        self.severity_predictor = joblib.load(f"{{model_path_prefix}}/severity_predictor_{timestamp}.pkl")
        
        # Load preprocessors
        self.vuln_scaler = joblib.load(f"{{model_path_prefix}}/scaler_vulnerability_{timestamp}.pkl")
        self.severity_vectorizer = joblib.load(f"{{model_path_prefix}}/vectorizer_severity_{timestamp}.pkl")
        self.vuln_encoder = joblib.load(f"{{model_path_prefix}}/encoder_vulnerability_{timestamp}.pkl")
        self.severity_encoder = joblib.load(f"{{model_path_prefix}}/encoder_severity_{timestamp}.pkl")
    
    def extract_code_features(self, solidity_code: str) -> np.ndarray:
        """Extract features from Solidity code"""
        
        # Basic code metrics
        line_count = len(solidity_code.split('\\n'))
        function_count = len(re.findall(r'function\\s+\\w+', solidity_code))
        modifier_count = len(re.findall(r'modifier\\s+\\w+', solidity_code))
        external_calls = len(re.findall(r'\\.call\\(|\\.delegatecall\\(|\\.send\\(|\\.transfer\\(', solidity_code))
        state_variables = len(re.findall(r'(mapping|uint256|address|bool)\\s+(public|private|internal)', solidity_code))
        require_statements = len(re.findall(r'require\\(', solidity_code))
        assembly_blocks = len(re.findall(r'assembly\\s*\\{{', solidity_code))
        event_declarations = len(re.findall(r'event\\s+\\w+', solidity_code))
        payable_functions = len(re.findall(r'payable', solidity_code))
        external_functions = len(re.findall(r'function\\s+\\w+.*external', solidity_code))
        public_functions = len(re.findall(r'function\\s+\\w+.*public', solidity_code))
        loops = len(re.findall(r'for\\s*\\(|while\\s*\\(', solidity_code))
        
        # Advanced features
        complexity_score = min(1.0, line_count / 100.0)
        has_fallback = 1 if 'fallback' in solidity_code else 0
        has_receive = 1 if 'receive' in solidity_code else 0
        uses_msg_sender = len(re.findall(r'msg\\.sender', solidity_code))
        uses_block_timestamp = len(re.findall(r'block\\.timestamp|now', solidity_code))
        uses_tx_origin = len(re.findall(r'tx\\.origin', solidity_code))
        
        # Ratios and derived features
        security_ratio = require_statements / max(function_count, 1)
        external_call_ratio = external_calls / max(function_count, 1)
        uses_assembly = 1 if assembly_blocks > 0 else 0
        code_length = len(solidity_code)
        function_density = function_count / max(line_count, 1)
        
        # Default protocol/contract assumptions for unknown contracts
        tier1, tier2, tier3 = 0, 1, 0  # Assume tier2
        token_type, defi_type, infra_type, gov_type = 0, 1, 0, 0  # Assume DeFi
        
        feature_vector = [
            line_count, function_count, modifier_count, external_calls,
            state_variables, require_statements, assembly_blocks, event_declarations,
            payable_functions, external_functions, public_functions, loops,
            complexity_score, 7.0,  # default CVE score
            tier1, tier2, tier3,
            token_type, defi_type, infra_type, gov_type,
            has_fallback, has_receive, uses_msg_sender, uses_block_timestamp, uses_tx_origin,
            security_ratio, external_call_ratio, uses_assembly,
            code_length, function_density
        ]
        
        return np.array(feature_vector).reshape(1, -1)
    
    def detect_vulnerabilities(self, solidity_code: str, description: str = "") -> Dict:
        """Detect vulnerabilities in smart contract code"""
        
        # Extract features
        features = self.extract_code_features(solidity_code)
        
        # Scale features
        features_scaled = self.vuln_scaler.transform(features)
        
        # Predict vulnerability type
        vuln_pred_encoded = self.vuln_classifier.predict(features_scaled)[0]
        vuln_prediction = self.vuln_encoder.inverse_transform([vuln_pred_encoded])[0]
        
        # Get prediction confidence
        vuln_proba = self.vuln_classifier.predict_proba(features_scaled)[0]
        vuln_confidence = float(np.max(vuln_proba))
        
        # Predict severity
        text_input = f"{{solidity_code}} {{description}}"
        text_features = self.severity_vectorizer.transform([text_input])
        severity_pred_encoded = self.severity_predictor.predict(text_features.toarray())[0]
        severity_prediction = self.severity_encoder.inverse_transform([severity_pred_encoded])[0]
        
        # Get severity confidence
        severity_proba = self.severity_predictor.predict_proba(text_features.toarray())[0]
        severity_confidence = float(np.max(severity_proba))
        
        # Calculate risk score
        severity_scores = {{'Critical': 1.0, 'High': 0.8, 'Medium': 0.6, 'Low': 0.4}}
        risk_score = severity_scores.get(severity_prediction, 0.5) * vuln_confidence
        
        return {{
            'vulnerability_type': vuln_prediction,
            'vulnerability_confidence': vuln_confidence,
            'severity_level': severity_prediction,
            'severity_confidence': severity_confidence,
            'risk_score': risk_score,
            'recommendations': self._get_recommendations(vuln_prediction, severity_prediction)
        }}
    
    def _get_recommendations(self, vulnerability: str, severity: str) -> str:
        """Get security recommendations based on detected vulnerability"""
        
        recommendations = {{
            'reentrancy': "Use checks-effects-interactions pattern. Implement reentrancy guards (nonReentrant modifier).",
            'integer_overflow': "Use SafeMath library or Solidity 0.8+ with built-in overflow checks.",
            'unchecked_call': "Always check return values of external calls. Use require() statements.",
            'access_control': "Implement proper access control with onlyOwner or role-based modifiers.",
            'timestamp_dependence': "Avoid using block.timestamp for critical logic. Use block numbers or external oracles.",
            'delegatecall_injection': "Validate implementation addresses. Use upgradeable proxy patterns carefully.",
            'tx_origin': "Use msg.sender instead of tx.origin for authorization checks.",
            'uninitialized_storage': "Always initialize storage pointers explicitly.",
            'random_weakness': "Use commit-reveal schemes or external randomness oracles (Chainlink VRF).",
            'dos_gas_limit': "Avoid unbounded loops. Implement pagination for large data sets."
        }}
        
        return recommendations.get(vulnerability, "Conduct thorough security audit and testing.")

# Example usage:
# detector = SmartContractVulnDetector("/path/to/models")
# 
# vulnerable_code = """
# function withdraw(uint256 amount) external {{
#     require(balances[msg.sender] >= amount);
#     (bool success, ) = msg.sender.call{{value: amount}}("");
#     require(success);
#     balances[msg.sender] -= amount;
# }}
# """
# 
# result = detector.detect_vulnerabilities(vulnerable_code)
# print(f"Vulnerability: {{result['vulnerability_type']}} ({{result['vulnerability_confidence']:.2f}})")
# print(f"Severity: {{result['severity_level']}} ({{result['severity_confidence']:.2f}})")
# print(f"Risk Score: {{result['risk_score']:.2f}}")
# print(f"Recommendations: {{result['recommendations']}}")
'''

# Save deployment code
deploy_path = sc_trainer.results_path / f"smart_contract_deployment_{timestamp}.py"
with open(deploy_path, 'w') as f:
    f.write(deployment_code)

print(f"\n📋 Smart Contract Deployment Code: {deploy_path}")

# Final summary
print(f"\n🎉 SMART CONTRACT VULNERABILITY TRAINING COMPLETE!")
print(f"=" * 60)
print(f"🤖 Vulnerability Classifier: {vuln_results['best_model']} (Acc={vuln_results['cv_accuracy_mean']:.3f})")
print(f"🎯 Severity Predictor: {severity_results['model_type']} (Acc={severity_results['cv_accuracy_mean']:.3f})")
print(f"📊 Dataset: {len(sc_df):,} smart contract samples")
print(f"🔐 Vulnerabilities: {len(vuln_results['class_names'])} types")
print(f"💾 Models saved with timestamp: {timestamp}")
print(f"\n🚀 Ready for smart contract security analysis!")
print(f"\n📖 Supported Vulnerabilities:")
for vuln in sc_trainer.vulnerability_categories.keys():
    severity = sc_trainer.vulnerability_categories[vuln]['severity']
    print(f"  • {vuln.replace('_', ' ').title()} ({severity})")